In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE

from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.model_selection import KFold

## Facebook Comment Volume Dataset

In [ ]:
df = pd.read_csv('datasets/facebook_comments.csv', index_col=0)

In [ ]:
print('dataset size: {}'.format(df.shape))

In [ ]:
# encoding categorical variables
# H Local - category
# Post Promotion Status - category
# Base Time - time variable
# Page Category - category

H_Local = OneHotEncoder().fit_transform(df['H Local'].values.reshape(-1, 1)).todense()
Post_Promotion_Status = OneHotEncoder().fit_transform(df['Post Promotion Status'].values.reshape(-1, 1)).todense()
Base_Time = OneHotEncoder().fit_transform(df['Base Time'].values.reshape(-1, 1)).todense()
Page_Category = OneHotEncoder().fit_transform(df['Page Category'].values.reshape(-1, 1)).todense()

In [ ]:
y = df.Target.values.reshape(-1, 1)
X = df.drop(['H Local', 'Post Promotion Status', 'Base Time', 'Page Category', 'Target'], axis=1).values
X = np.hstack([X, H_Local, Post_Promotion_Status, Base_Time, Page_Category])

In [ ]:
print('dataset size after preprocessing: {}'.format(X.shape))

In [ ]:
# number of components is fixed to 3
# it caused by the limitation of Barmes-Hut algorithm
# since exact calculations take significant time/memory resource
tsne_cases = []

for i in range(5):
    print('{}-th iteration'.format(i))
    tsne = TSNE(n_components=3, method='barnes_hut', random_state=i * 5)
    tsne_cases.append(tsne.fit_transform(X))

In [ ]:
X, y = shuffle(X, y)

models = {
    'linear': LinearRegression(),
    'forest': LGBMRegressor()
}

stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X, y):
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        
        model.fit(X_train, y_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_test), y_test))

In [ ]:
print('for pure data:')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

In [ ]:
models = {
    'linear': LinearRegression(),
    'forest': LGBMRegressor()
}

stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for X_tsne in tsne_cases:
        X, y = shuffle(X_tsne, y)
        for train_ix, test_ix in kfold.split(X, y):
            X_train, X_test = X[train_ix], X[test_ix]
            y_train, y_test = y[train_ix], y[test_ix]

            model.fit(X_train, y_train.ravel())
            stats[k].append(mean_squared_error(model.predict(X_test), y_test))
        

In [ ]:
print('for TSNE-processed data')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

## Parkinsons Telemonitoring Data Set

In [ ]:
df = pd.read_csv('datasets/parkinsons_updrs.data')

In [ ]:
print('dataset size: {}'.format(df.shape))

In [ ]:
subject = df['subject#'].unique().tolist()
subject_binary = np.zeros((df.shape[0], len(subject)))
for k, i in df.iterrows():
    subject_binary[k, int(i['subject#']) - 1] = 1

In [ ]:
# replacing categorical features with binary values
y = df.total_UPDRS
X = df.drop(['motor_UPDRS', 'total_UPDRS', 'subject#'], axis=1).values
X = np.concatenate([subject_binary, X], axis=1)

In [ ]:
print('dataset size after preprocessing: {}'.format(X.shape))

In [ ]:
# number of components is fixed to 3
tsne_cases = []

for i in range(5):
    print('{}-th iteration'.format(i))
    tsne = TSNE(n_components=3, method='barnes_hut', random_state=i * 5)
    tsne_cases.append(tsne.fit_transform(X))

In [ ]:
X, y = shuffle(X, y)

models = {
    'linear': LinearRegression(),
    'forest': LGBMRegressor()
}

stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X, y):
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        
        model.fit(X_train, y_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_test), y_test))

In [ ]:
print('for pure data:')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

In [ ]:
models = {
    'linear': LinearRegression(),
    'forest': LGBMRegressor()
}

stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for X_tsne in tsne_cases:
        X, y = shuffle(X_tsne, y)
        for train_ix, test_ix in kfold.split(X, y):
            X_train, X_test = X[train_ix], X[test_ix]
            y_train, y_test = y[train_ix], y[test_ix]

            model.fit(X_train, y_train.ravel())
            stats[k].append(mean_squared_error(model.predict(X_test), y_test))
        

In [ ]:
print('for TSNE-processed data')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

## Energy efficiency Data Set

In [3]:
df = pd.read_excel('./datasets/ENB2012_data.xlsx')

In [4]:
print('dataset size: {}'.format(df.shape))

dataset size: (768, 10)


In [5]:
subject = df['X6'].unique().tolist()
subject_map = dict(zip(subject, range(len(subject))))
subject_binary = np.zeros((df.shape[0], len(subject)))
for k, i in df.iterrows():
    subject_binary[k, subject_map[i['X6']]] = 1

In [6]:
# replacing categorical features with binary values
y = df.Y1
X = df.drop(['Y1', 'Y2', 'X6'], axis=1).values
X = np.concatenate([subject_binary, X], axis=1)

In [7]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (768, 11)


In [8]:
# number of components is fixed to 3
tsne_cases = []

for i in range(5):
    print('{}-th iteration'.format(i))
    tsne = TSNE(n_components=3, method='barnes_hut', random_state=i * 5)
    tsne_cases.append(tsne.fit_transform(X))

0-th iteration
1-th iteration
2-th iteration
3-th iteration
4-th iteration


In [ ]:
X, y = shuffle(X, y)

models = {
    'linear': LinearRegression(),
    'forest': LGBMRegressor()
}

stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X, y):
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        
        model.fit(X_train, y_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_test), y_test))

In [ ]:
print('for pure data:')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

In [ ]:
models = {
    'linear': LinearRegression(),
    'forest': LGBMRegressor()
}

stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for X_tsne in tsne_cases:
        X, y = shuffle(X_tsne, y)
        for train_ix, test_ix in kfold.split(X, y):
            X_train, X_test = X[train_ix], X[test_ix]
            y_train, y_test = y[train_ix], y[test_ix]

            model.fit(X_train, y_train.ravel())
            stats[k].append(mean_squared_error(model.predict(X_test), y_test))
        

In [ ]:
print('for TSNE-processed data')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))